In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




```
Calculate Customer Lifetime Value (CLTV = ((Average Order Value x Purchase Frequency / Churn Rate) x Profit margin
```





In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Ecommerce_Data-1.csv")
df.head(3)

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour
0,439570,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453.0,United Kingdom,2011-11-04,12
1,387281,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098.0,United Kingdom,2011-10-10,10
2,337863,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609.0,United Kingdom,2011-09-13,9


In [3]:
df['total_revenue'] = df['Quantity']*df['UnitPrice']
df.drop('Unnamed: 0', axis=1, inplace=True)
df['Date']=pd.to_datetime(df['Date'])
df = df[df['Quantity']>0]
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country,Date,Hour,total_revenue
0,574477,22591,CARDHOLDER GINGHAM CHRISTMAS TREE,1,3.25,15453.0,United Kingdom,2011-11-04,12,3.25
1,570275,23541,WALL ART CLASSIC PUDDINGS,12,7.45,13098.0,United Kingdom,2011-10-10,10,89.40
2,566482,22508,DOORSTOP RETROSPOT HEART,12,3.75,16609.0,United Kingdom,2011-09-13,9,45.00


In [5]:
max_date = df['Date'].max()
df2 = df.groupby('CustomerID').agg(
    {'Date':lambda x: (max_date-x.min()).days,
     'InvoiceNo': lambda x:len(x),
     'Quantity': lambda x: x.sum(),
     'total_revenue': lambda x:x.sum()})
df2.head(2)

,Date,InvoiceNo,Quantity,total_revenue
CustomerID,,,,
12347.0,367,48,623,1146.59
12348.0,358,7,531,550.92


In [6]:
max_date

Timestamp('2011-12-09 00:00:00')

In [7]:
def customer_model(data):
  max_date = data['Date'].max()
  data = data.groupby('CustomerID').agg(
      {'Date':lambda x: (max_date-x.min()).days,
      'InvoiceNo': lambda x:len(x),
      'Quantity': lambda x: x.sum(),
      'total_revenue': lambda x:x.sum()})
  return data

data = customer_model(df)
data.head(2)

,Date,InvoiceNo,Quantity,total_revenue
CustomerID,,,,
12347.0,367,48,623,1146.59
12348.0,358,7,531,550.92


In [8]:
data.columns=['age', 'num_trans', 'quantity', 'total_revenue']
data = data[data['quantity']>0]
data

,age,num_trans,quantity,total_revenue
CustomerID,,,,
12347.0,367,48,623,1146.59
12348.0,358,7,531,550.92
12349.0,18,16,203,406.32
12350.0,310,2,36,45.60
12352.0,296,21,163,663.33
...,...,...,...,...
18280.0,277,3,9,52.75
18281.0,180,2,11,33.45
18282.0,126,4,11,67.85


# Get the element of CLTV Calculation

In [9]:
data['AOV'] = data['total_revenue']/data['num_trans']
data

,age,num_trans,quantity,total_revenue,AOV
CustomerID,,,,,
12347.0,367,48,623,1146.59,23.887292
12348.0,358,7,531,550.92,78.702857
12349.0,18,16,203,406.32,25.395000
12350.0,310,2,36,45.60,22.800000
12352.0,296,21,163,663.33,31.587143
...,...,...,...,...,...
18280.0,277,3,9,52.75,17.583333
18281.0,180,2,11,33.45,16.725000
18282.0,126,4,11,67.85,16.962500


In [10]:
purchase_freq = sum(data['num_trans'])/len(data)
purchase_freq

19.496336101612115

In [11]:
repeat_rate = data[data['num_trans']>1].shape[0]/data.shape[0]
repeat_rate

0.9291646311675623

In [12]:
churn_rate = 1 - repeat_rate
churn_rate

0.07083536883243768

In [13]:
data['profit_margin'] = data['total_revenue']*.10

In [14]:
data.head(4)

,age,num_trans,quantity,total_revenue,AOV,profit_margin
CustomerID,,,,,,
12347.0,367,48,623,1146.59,23.887292,114.659
12348.0,358,7,531,550.92,78.702857,55.092
12349.0,18,16,203,406.32,25.395000,40.632
12350.0,310,2,36,45.60,22.800000,4.560


# Put the equation together

In [15]:
data['CLTV'] = ((data['AOV']*purchase_freq)/churn_rate)*.10
data

,age,num_trans,quantity,total_revenue,AOV,profit_margin,CLTV
CustomerID,,,,,,,
12347.0,367,48,623,1146.59,23.887292,114.659,657.460637
12348.0,358,7,531,550.92,78.702857,55.092,2166.174018
12349.0,18,16,203,406.32,25.395000,40.632,698.957969
12350.0,310,2,36,45.60,22.800000,4.560,627.534621
12352.0,296,21,163,663.33,31.587143,66.333,869.387093
...,...,...,...,...,...,...,...
18280.0,277,3,9,52.75,17.583333,5.275,483.953966
18281.0,180,2,11,33.45,16.725000,3.345,460.329672
18282.0,126,4,11,67.85,16.962500,6.785,466.866491


In [16]:
import seaborn
print('the median CLTV is ', data['CLTV'].median())
print('the average CLTV is ', data['CLTV'].mean())

the median CLTV is  460.5410131773401
the average CLTV is  747.7434697480141


In [17]:
data

,age,num_trans,quantity,total_revenue,AOV,profit_margin,CLTV
CustomerID,,,,,,,
12347.0,367,48,623,1146.59,23.887292,114.659,657.460637
12348.0,358,7,531,550.92,78.702857,55.092,2166.174018
12349.0,18,16,203,406.32,25.395000,40.632,698.957969
12350.0,310,2,36,45.60,22.800000,4.560,627.534621
12352.0,296,21,163,663.33,31.587143,66.333,869.387093
...,...,...,...,...,...,...,...
18280.0,277,3,9,52.75,17.583333,5.275,483.953966
18281.0,180,2,11,33.45,16.725000,3.345,460.329672
18282.0,126,4,11,67.85,16.962500,6.785,466.866491


In [18]:
data.sort_values(by='CLTV').reset_index()

,CustomerID,age,num_trans,quantity,total_revenue,AOV,profit_margin,CLTV
0,12977.0,156,1,2,0.78,0.780000,0.078,21.468290
1,14661.0,26,1,1,0.85,0.850000,0.085,23.394931
2,16658.0,368,1,1,0.85,0.850000,0.085,23.394931
3,17962.0,142,3,3,2.99,0.996667,0.299,27.431703
4,17859.0,304,5,5,5.21,1.042000,0.521,28.679433
...,...,...,...,...,...,...,...,...
4089,12931.0,239,15,5590,9344.82,622.988000,934.482,17146.777994
4090,16333.0,298,11,7840,7944.40,722.218182,794.440,19877.934771
4091,17949.0,289,10,4565,7671.45,767.145000,767.145,21114.475728
4092,16000.0,2,5,2630,7948.30,1589.660000,794.830,43752.924786
